# グラフ&フロー  
Source:始点, sink:終点  
## 最大流問題  
あるノードから別のノードへの流量の最大値を求める  
・貪欲法(greedy algorithm) 今ある選択肢のうち，なにか1つを選んで次に進む 必ずしも最適解を導出できるとは限らない  
経路をDFSで見つけて流せるだけ流す　が、探索しきれない  
・フォード・ファルカーソン法  
基本は貪欲法と同じだが逆方向の経路も想定する   
ノード間を逆にたどる経路を考えることで，流しすぎたフローを逆に戻す処理を実現している  
##### これによって，流す経路を導出するのをDFSに⼀本化できている！


In [2]:
#実装例
capacity =[ [0, 10, 10, 0, 0, 0], # A->B: 10,  A->C: 10 
           [0, 0, 0, 4, 8, 0], # B->D: 4,  B->E: 8 
           [0, 0, 0, 7, 4, 0], # C->D: 7,  C->E: 4 
           [0, 0, 0, 0, 0, 8], # D->F: 8 
           [0, 0, 0, 0, 0, 12], # E->F: 12
           [0, 0, 0, 0, 0, 0] # Fから出ていくものなし
           ]
max_flow=0
V=6
#s-e間において流量を返す
def dfs_ff(s,e,flow):
    if s==e:
        return flow
    visited[s]=True
    for i in range(V):
        if not visited[i] and capacity[s][i]>0:
            next_flow=min(flow,capacity[s][i])
            f=dfs_ff(i,e,next_flow)
            if f>0:
                capacity[s][i]-=f
                capacity[i][s]+=f
                return f
    return 0

while True:
    visited=[False for _ in range(V)]
    f=dfs_ff(0,5,10**9)
    if not f:
        break
    max_flow+=f
print(max_flow)

20


### フォード・ファルカーソン法の時間計算量  
ノード数$|V|$, 最大流の最適値をFとする  
最悪の場合、新しい経路が見つかっても最大流が1つしか増やせない→F回の最大流値の更新が発生  
新しい経路を探すための操作 $O(|V|)$より上記実装方法では$O(F|V|^2)$となる  
一方で隣接リストなら$O(F|E|)$となる  
ただし経路の容量が整数もしくは有理数である必要があり、無理数の場合は最適解に収束しない可能性がある。  
また、最大流量Fは未知数であり、実行してみないとわからない。→ エドモンズ・カープ法：$O(|V||E|^2)$, ディニッツ法：$O(|V|)^2|E|$

## グラフのカット
カット＝あるグラフを2つのグループ（集合）に分ける  
s-tカット : 𝑆に開始ノードが，𝑉\𝑆に終点ノードが含まれるカット  
フロー𝑓は以下のように計算される．𝑓 =[𝑆から𝑉∖𝑆への流⼊量]−[𝑉∖𝑆から𝑆への流⼊量] → 𝑓 ≤ [𝑆から𝑉∖𝑆への流⼊量]  
さらに，[𝑆から𝑉∖𝑆への流⼊量]はそのパスの容量を超えることはない．よって，𝑓 ≤ [𝑆から𝑉∖𝑆への流⼊量] ≤ 𝑈(𝑆→𝑉∖𝑆)  
##### 最小カット
s-tカットにおいて，𝑆から𝑉∖𝑆へ向かう辺の容量の和𝑈(𝑆 → 𝑉∖𝑆)が最⼩になるようなカット．  
<最大流・最小カット定理>  
「与えられたグラフにおいて，最⼤流の流量はs-tカットにおける最⼩カットの容量に等しい」  
[ノードに容量がある場合]  
ノードを⼊ってくる⽅と出る⽅の2つに分けて，その間をノードの容量で結ぶ．  
[複数のsourceやsinkがある場合]  
各sourceに向かう辺を持つ新しい始点ノードを作り，その各辺にそれぞれの最⼤流出量を付与する．sinkの場合も同様に新しい終点ノードを作れば良い．

### 計算量
s-tカットの分け方：ファード・ファルカーソン法により$O(F|V|^2)$や$O(F|E|)$で求められる  


#### フローグラフの典型的な応用
二部グラフ：頂点集合を2つの部分集合に分割でき，各集合内の頂点同⼠の間には辺が無いグラフ  
最⼤マッチングペアの数を最⼤にする辺の組み合わせを考える。これを最大流問題に置き換えると  
辺の容量を1として，左のグループから右のグループに流れる流量が最⼤になる辺の組み合わせを考える．全ての辺の容量は1なので，流れる流量が最⼤になれば，左のグループから右のグループへと流すために使った辺の数も最⼤，つまり左右の組み合わせ数も最⼤となっている状態を作り出せる


In [2]:
#基本課題
N,M=map(int,input().split())
List=[list(map(int,input().split())) for _ in range(M)]

def make_edges_list(L):
    edges_list=[[0]*N for _ in range(N)]
    for u,v,c in L:
        edges_list[u-1][v-1]=c
    return edges_list
capacity=make_edges_list(List)
max_flow=0

def dfs_ff(s,e,flow):
    if s==e:
        return flow
    visited[s]=True
    for i in range(N):
        if not visited[i] and capacity[s][i]>0:
            next_flow=min(flow,capacity[s][i])
            f=dfs_ff(i,e,next_flow)
            if f>0:
                capacity[s][i]-=f
                capacity[i][s]+=f
                return f
    return 0

while True:
    visited=[False for _ in range(N)]
    f=dfs_ff(0,N-1,10**9)
    if not f:
        break
    max_flow+=f
print(max_flow)

[[1, 2, 3], [1, 3, 1], [1, 4, 1], [2, 3, 2], [3, 4, 2], [2, 4, 1]]
4


In [ ]:
N,M,H=map(int,input().split())
List=[list(map(int,input().split())) for _ in range(M)]

In [22]:

def make_edges_list(L,h):
    edges_list=[[0]*N for _ in range(N)]
    for u,v,c in L:
        edges_list[u-1][v-1]=c-h
    return edges_list

def dfs_ff(s,e,flow,visited,capacity):
    if s==e:
        return flow
    visited[s]=True
    for i in range(N):
        if not visited[i] and capacity[s][i]>0:
            next_flow=min(flow,capacity[s][i])
            f=dfs_ff(i,e,next_flow,visited,capacity)
            if f>0:
                capacity[s][i]-=f
                capacity[i][s]+=f
                return f
    return 0
def calc_cost(capacity):
    max_flow=0
    while True:
        visited=[False for _ in range(N)]
        f=dfs_ff(0,N-1,10**9,visited,capacity)
        if not f:
            break
        max_flow+=f
    return max_flow

for h in range(H):
    capacity=make_edges_list(List,h)
    print(calc_cost(capacity))

4
2
1
0


In [21]:
# エッジのリストを基にフロー容量の行列を作成する関数
def make_edges_list(L):
    edges_list = [[0]*N for _ in range(N)]
    for u, v, c in L:
        edges_list[u-1][v-1] = c
    return edges_list

# s から e までのフローをDFSで計算する関数
def dfs_ff(s, e, flow, visited,capacity):
    if s == e:
        return flow
    visited[s] = True
    for i in range(N):
        if not visited[i] and capacity[s][i] > 0:
            next_flow = min(flow, capacity[s][i])
            f = dfs_ff(i, e, next_flow, visited,capacity)
            if f > 0:
                capacity[s][i] -= f
                capacity[i][s] += f
                return f
    return 0

# 最大フローを計算する関数
def calc_cost():
    max_flow=0
    local_capacity = [row[:] for row in capacity]
    while True:
        visited = [False for _ in range(N)]
        f = dfs_ff(0, N-1, 10**9, visited,local_capacity)
        if not f:
            break
        max_flow += f
    return max_flow


# フロー容量の行列を作成
capacity = make_edges_list(List)

# 繰り返し計算（例では1回）
for h in range(H):
    print(calc_cost())


4
4
4
4
